In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torch.autograd import Variable

import pandas as pd
import numpy as np

In [10]:
class mydataset(data_utils.Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path, nrows = 10000)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data.iloc[index, 1:].values.astype(np.float32)
        Y = self.data.iloc[index, 0].astype(np.float32)
        
        return X, Y

In [11]:
training = mydataset('HIGGS.csv')
train_loader = data_utils.DataLoader(training, batch_size=1, shuffle=F)

In [12]:
# layer_dims (batch_size, input_dim, hidden_layers, output_dims) 
layer_dims = (10000, 28, 15, 15, 1)
#device = torch.device("cuda:0")

In [13]:
class Net(nn.Module):
    def __init__(self, layer_dims):
        super(Net, self).__init__()
        
        self.linear1 = nn.Linear(layer_dims[1], layer_dims[2])
        self.linear1_out = nn.Linear(layer_dims[2], 1)
        self.linear2 = nn.Linear(layer_dims[2], layer_dims[3])
        self.linear2_out = nn.Linear(layer_dims[3], 1)
        self.linear3 = nn.Linear(layer_dims[3], layer_dims[4])
        self.linear3_out = nn.Linear(layer_dims[4], 1)
        #self.linear4 = nn.Linear(layer_dims[4], layer_dims[5])
        #self.linear4_out = nn.Linear(layer_dims[5], 1)
        #self.linear5 = nn.Linear(layer_dims[5], layer_dims[6])
        #self.linear5_out = nn.Linear(layer_dims[6], 1)
        #self.linear6 = nn.Linear(layer_dims[6], layer_dims[7])
        #self.linear6_out = nn.Linear(layer_dims[7], 1)
        #self.linear7 = nn.Linear(layer_dims[7], layer_dims[8])
        #self.linear7_out = nn.Linear(layer_dims[8], 1)
        #self.linear8 = nn.Linear(layer_dims[8], layer_dims[9])
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        

        
        layer1 = self.linear1(x)
        layer1_act = self.relu(layer1)
        layer1_out = self.sigmoid(self.linear1_out(layer1_act)) 
        
        layer2 = self.linear2(layer1_act)
        layer2_act = self.relu(layer2) 
        layer2_out = self.sigmoid(self.linear2_out(layer2_act))
        
        layer3 = self.linear3(layer2_act)
        layer3_act = self.relu(layer3) 
        layer3_out = self.sigmoid(self.linear3_out(layer3_act))
        
        #layer4 = self.linear4(layer3_act)
        #layer4_act = self.relu(layer4) 
        #layer4_out = self.sigmoid(self.linear4_out(layer4_act))
        
        #layer5 = self.linear5(layer4_act)
        #layer5_act = self.relu(layer5) 
        #layer5_out = self.sigmoid(self.linear5_out(layer5_act))
        
        #layer6 = self.linear6(layer5_act)
        #layer6_act = self.relu(layer6) 
        #layer6_out = self.sigmoid(self.linear6_out(layer6_act))
        
        #layer7 = self.linear7(layer6_act)
        #layer7_act = self.relu(layer7) 
        #layer7_out = self.sigmoid(self.linear7_out(layer7_act))
        
        #layer8 = self.linear8(layer7_act)
        #layer8_out = self.sigmoid(layer8)
        
        return layer1_out,layer2_out, layer3_out                                    

In [14]:
model = Net(layer_dims)
print(model)

optimizer = optim.Adam(model.parameters(), lr= .01)
loss_function = nn.BCELoss()

Net(
  (linear1): Linear(in_features=28, out_features=15, bias=True)
  (linear1_out): Linear(in_features=15, out_features=1, bias=True)
  (linear2): Linear(in_features=15, out_features=15, bias=True)
  (linear2_out): Linear(in_features=15, out_features=1, bias=True)
  (linear3): Linear(in_features=15, out_features=1, bias=True)
  (linear3_out): Linear(in_features=1, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [15]:
model.train()

Net(
  (linear1): Linear(in_features=28, out_features=15, bias=True)
  (linear1_out): Linear(in_features=15, out_features=1, bias=True)
  (linear2): Linear(in_features=15, out_features=15, bias=True)
  (linear2_out): Linear(in_features=15, out_features=1, bias=True)
  (linear3): Linear(in_features=15, out_features=1, bias=True)
  (linear3_out): Linear(in_features=1, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [19]:
alpha = [1/(len(layer_dims)-2)] * (len(layer_dims)-2)

for i in range(1):
    
    for batch_idx, data in enumerate(train_loader):
        
            x_train = data[0]
            y_train = data[1]
    
            out1, out2, out3 = model(x_train)

            losses = []

            loss1 = loss_function(out1, y_train) 
            loss2 = loss_function(out2, y_train) 
            loss3 = loss_function(out3, y_train) 
            #loss4 = loss_function(out4, y_train) * alpha[3]
            #loss5 = loss_function(out5, y_train) * alpha[4]
            #loss6 = loss_function(out6, y_train) * alpha[5]
            #loss7 = loss_function(out7, y_train) * alpha[6]
            #loss8 = loss_function(out8, y_train) * alpha[7]

            loss = loss1 * alpha[0] + loss2 * alpha[1] + loss3 * alpha[2] 

            losses.append(loss1.item());losses.append(loss2.item());losses.append(loss3.item());
            #losses.append(loss5.item());losses.append(loss6.item());losses.append(loss7.item());losses.append(loss8.item());
            M = sum(losses)
            losses = [loss / M for loss in losses]
            min_loss = np.amin(losses)
            max_loss = np.amax(losses)
            range_loss = max_loss - min_loss

            losses = [(loss-min_loss)/range_loss for loss in losses]
            alpha_u = [0.99 ** loss for loss in losses]
            alpha = [a * w for a, w in zip(alpha_u, alpha)]

            alpha = [ max(0.01, a) for a in alpha]
            M = sum(alpha)
            alpha = [a / M for a in alpha]


            if(batch_idx % 200 == 0):
                print(batch_idx, 'Total_loss',loss.item(), 
                      'Loss1:', loss1.item(),
                      'Loss2:', loss2.item(),
                      'Loss3:', loss3.item())
                      #'Loss4:', loss4.item(),
                      #'Loss5:', loss5.item(),
                      #'Loss6:', loss6.item(),
                      #'Loss7:', loss7.item(),
                      #'Loss8:', loss8.item())
                print('alpha:', alpha)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()




C:\Mrugank_pc\Softwares\Python\Anaconda3_5.1.0\lib\site-packages\torch\nn\functional.py:2016: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


0 Total_loss 0.6372918486595154 Loss1: 0.6380852460861206 Loss2: 0.6369341015815735 Loss3: 0.6368561387062073
alpha: [0.33117426606675937, 0.3343062732597462, 0.33451946067349436]
200 Total_loss 0.6658570766448975 Loss1: 0.6658321619033813 Loss2: 0.6658939719200134 Loss3: 0.6658475995063782
alpha: [0.35352915473971386, 0.3194472408494711, 0.3270236044108151]
400 Total_loss 0.6088424324989319 Loss1: 0.6087393760681152 Loss2: 0.6098716259002686 Loss3: 0.6080480813980103
alpha: [0.35395778086121693, 0.29977656974607825, 0.3462656493927049]
600 Total_loss 0.7815289497375488 Loss1: 0.782187283039093 Loss2: 0.7787351012229919 Loss3: 0.7831317186355591
alpha: [0.36130134974250544, 0.28865911273656925, 0.3500395375209253]
800 Total_loss 0.6400877237319946 Loss1: 0.6399809718132019 Loss2: 0.640520453453064 Loss3: 0.6398199200630188
alpha: [0.35847697792235916, 0.29814787088910943, 0.3433751511885314]
1000 Total_loss 0.8029677867889404 Loss1: 0.8032190799713135 Loss2: 0.801891028881073 Loss3: 0.

9200 Total_loss 0.664607048034668 Loss1: 0.6653962135314941 Loss2: 0.6672605276107788 Loss3: 0.6619144082069397
alpha: [0.31674742010772644, 0.2955907117413913, 0.38766186815088227]
9400 Total_loss 0.7344732880592346 Loss1: 0.7348304390907288 Loss2: 0.7354527711868286 Loss3: 0.7334564328193665
alpha: [0.31393001975349716, 0.29145410160250806, 0.39461587864399483]
9600 Total_loss 0.6147271394729614 Loss1: 0.6149125695228577 Loss2: 0.6147071123123169 Loss3: 0.6145911812782288
alpha: [0.30653379795414504, 0.3181776126177105, 0.3752885894281446]
9800 Total_loss 0.6089295744895935 Loss1: 0.607872486114502 Loss2: 0.6092247366905212 Loss3: 0.6095185279846191
alpha: [0.30289776110740796, 0.31787694138661576, 0.3792252975059762]


In [ ]:
model.eval()